# install libraries

In [ ]:
#instll this library - read the doc 
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install python-Levenshtein # install this to make fuzzy matching faster

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import libraries

In [ ]:
#Import pandas
import pandas as pd

#Impor regex
import re

#Import csv
import csv

from IPython.utils.text import strip_email_quotes

from fuzzywuzzy import process, fuzz

# Choose folder to work on 

In [ ]:
#Insert the folder you are working on (e.g. folder = '18 Living with Social Media') and the VCS number (e.g. vcs_number = '18')

folder = '21 Achieving Peak Performance'
vcs_number = '21'

### set up working directory

In [ ]:
##Connecting with Gdrive
from google.colab import drive
drive.mount('/content/drive')

#Import Module
import os
  
# Folder Path
path = f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}'

# Change the directory
os.chdir(path)

Mounted at /content/drive


# Question - answers from users 

##Clean .txt files  

In [ ]:
#Get a list of txt files inside the folder :
def file_list():
  global file_path_list
  file_path_list=[] 
  ## iterate through all files
  for file in os.listdir():
      # Check whether file is in text format or not
      if file.endswith('.txt'):
          file_path = f"{path}/{file}"
          file_path_list.append(file_path)

In [ ]:
def ignore_files_cleaned():
    global log_files_cleaned
    log_files_cleaned=[]# set to empty
    global file_path_list # set this variable to global 
    file_path = 'log_files_cleaned.csv'

    try:
        # Load the list from the CSV file
        with open(file_path, 'r') as csvfile:
            reader = csv.reader(csvfile)
            log_files_cleaned = list(reader)[0]

    #if the file is not found        
    except FileNotFoundError:
        print(f"File '{file_path}' not found.THIS MIGHT BE THE FIRST TIME FILES ARE CLEANED\n ALL .txt FILES WILL BE LOADED")

    # remove elements from file_path_list that were already cleaned
    file_path_list = [x for x in file_path_list if x not in log_files_cleaned]

In [ ]:
## FUNCTION opens a file, iterates over each line in the file, removes the newline character from each line, and appends the modified lines to a list.

def generate_script(file_path): 
  global script
  script=[] ## set script to empty
  with open(file_path, 'r') as f:
    for line in f:
      # remove newline character
      clean_line=line.rstrip('\n')
      #check if line is empty
      if clean_line !='':
         # append to list 
        script.append(clean_line)  

In [ ]:
#write log_files_cleaned to csv 
def write_list(l):
  file_path = 'log_files_cleaned.csv'  # Path to the output CSV file
  
   # Save the list to a CSV file
  with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(l)

### Search for Question-Answer line with regex  

In [ ]:

#Searching for questions & answers rows:
#If you print script you will notice that the rows with the questions, will be preceded by a ### 2022/07/05 06:45:00 | 18 Living with Social Media (Complete) ###; 
# and the answers row will be always be preceded by a ### 2022/07/05 06:44:58 | Visitor #189139665 ### hence the regular expression. 

def extract_q_a():
  global regexq
  global regexa
  global q_a
  global q_a_clean
  global first_match
  q_a=[] ## set q_a to empty 
  q_a_clean = [] ## set to empty 
  regexq = re.compile(r'\W{3}\s\d{4}\W\d{2}\W\d{2}\s\d{2}\W\d{2}\W\d{2}\s\W\s\d')  #Regular expression to signal a question row.
  regexa = re.compile(r'\W{3}\s\d{4}\W\d{2}\W\d{2}\s\d{2}\W\d{2}\W\d{2}\s\W\sVisitor') #Regular expression to signal a answer row.

  ### WE WILL ONLY START COUNTING AFTER THE FIRST QUESTION. TO AVOID THOSE LINES  WHEN A CHAT STARTS SUCH AS ('hello holi')

  first_match = False  # Flag to track the first match of regexq

  for index, row in enumerate(script):
      if not first_match and re.match(regexq, row):
          first_match = True  # Set the flag to True after the first match

      #if the line is a question :
      if first_match and re.match(regexq, row):
          q_a.append(row[4:23].strip()) # remove extra word and non word character
          q_a.append(script[index+1].strip())#The question or the answer, will be always next of the row with the regex.

      # if the line is an answer:
      if first_match and  re.match(regexa, row):
          q_a.append(row[35:45].strip()) # remove extra word and non word character
          q_a.append(script[index+1].strip()) #The question or the answer, will be always next of the row with the regex.
          
          # if the text of the row is an answer,we want to retrieve both the question and the answer:
          # generate list of lists [date,question,user,answer] 
          q_a_clean.append(q_a[-4:])  #We are creating a list of list with date, question, user number and answer)


### Iterating over all the txt files in the folder 

In [ ]:
file_list() # returns a list of files (file_path_list)
ignore_files_cleaned() # returns (file_path_list) after removing the paths that were inside log_of_files_cleaned. if it exists

#file_path_list #check the list returned

df_q_a=pd.DataFrame()

#loop over the list of files 

for file_path in (file_path_list):
  generate_script(file_path) # returns all the conversation (script)
  #add path to the log
  log_files_cleaned.append(file_path) 

  extract_q_a() # returns questions and answers  (q_a) and q_a_clean
  
  #write q_a_clean to a dataframe 
  # Create a temporary DataFrame for the current iteration's list
  temp_df = pd.DataFrame(q_a_clean, columns =['created_at', 'question','user_number','answer'])

  # concat the temporary DataFrame to the main DataFrame
  df_q_a = pd.concat([df_q_a,temp_df], ignore_index=True)


#write the list of file_path to the log in .csv
write_list(log_files_cleaned)

File 'log_files_cleaned.csv' not found.THIS MIGHT BE THE FIRST TIME FILES ARE CLEANED. Skipping file loading.
 ALL .txt FILES WILL BE LOADED


## CHECKPOINT questions-answers

### Save  Cleaned questions-answers  DataFrame

In [ ]:
# save dataframe (UNCOMENT THIS FOR SAVING )

#df_q_a.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_test_nahuel.csv')

### Read saved  questions-answers DataFrame

In [ ]:
# read saved dataFrame 

# Specify the file path
file_path = (f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_test_nahuel.csv')

# Read the CSV file into a DataFrame
df_q_a = pd.read_csv(file_path,index_col=0)

#remove trailing '.' from questions and andswers 
df_q_a['question']=df_q_a['question'].str.rstrip('. !')
df_q_a['answer']=df_q_a['answer'].str.rstrip('. !')

In [ ]:
# we can use a list of questions to filter by 

In [ ]:
question_list=[]

# Users data ( email-user number) 

make sure there is a .csv file in data processed with the name:

VCS{vcs_number}_users_db.csv

columns: @email , @id

### Read users data 

In [ ]:
#Importing user's info from Drive
users = pd.read_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/VCS{vcs_number}_users_db.csv')

users.rename(columns = {'@email':'user_email', '@id':'user_number'}, inplace = True)

users['user_number']=users['user_number'].astype(int)

##Join questions-answer to user data 

In [ ]:

# set user_number as string 
#df_q_a['user_number'] = df_q_a['user_number'].str.replace(r'[^0-9]', '',regex=True) ##uncoment or coment this as needed 
df_q_a['user_number'] = df_q_a['user_number'].astype(int)

NameError: ignored

In [ ]:
#Merging both dataframes
all_data = df_q_a.merge(users, on = 'user_number')

In [ ]:
all_data.sort_values('created_at',ascending=False)

,created_at,question,user_number,answer,user_mail
216,2022/12/12 06:13:35,https://storage.googleapis.com/media.landbot.i...,228048716,Continue,charityyy.xoxo@gmail.com
215,2022/12/12 06:12:49,https://storage.googleapis.com/media.landbot.i...,228048716,Continue,charityyy.xoxo@gmail.com
214,2022/12/12 06:11:47,Which of the following statements helps you be...,228048716,I found all of these statements helpful,charityyy.xoxo@gmail.com
213,2022/12/12 06:11:19,"<iframe width=""250"" height=""315"" src=""https://...",228048716,Continue,charityyy.xoxo@gmail.com
212,2022/12/12 06:10:30,"First, I’m curious… Why did you decide to take...",228048716,All of these statements explain why I want to ...,charityyy.xoxo@gmail.com
...,...,...,...,...,...
3379,2021/07/28 23:13:13,"Okay, let’s jump right in",117638351,Sounds good,je.suis.paheli03@gmail.com
1085,165848322,START AGAIN,165848322,Thank you Holi,joshua@solavieve.com
2547,143605515,Strongly disagree,143605515,👍,valoyitinyiko85@gmail.com
2551,143605515,All of these statements explain why I want to ...,143605515,Continue,valoyitinyiko85@gmail.com


# Question - Catalogue 

make sure there is a .csv file in data processed with the name:

question_catalogue

columns: question_v1 , answers_v1 ,	type , reverse_answer? [yes/no]

* in the column answers in the excel file . REPLACE a) b) c) etc with 1,2,3 etc .

* fill in wich health tracking questions are reverse or not: **If 1 is good > reverse = yes** 




## Read question_catalogue file 

In [ ]:
# read question_catalogue
# Specify the file path
file_path = (f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/question_catalogue.csv')

# Read the CSV file into a DataFrame
question_catalogue = pd.read_csv(file_path)

In [ ]:
question_catalogue.head(5)

,question_v1,answers_v1,type,reverse_answer? [yes/no]
0,"First, I’m curious… Why did you decide to tak...",1.I’m curious about the concept of peak perfor...,General,NaN
1,Which of the following statements helps you be...,1.Peak performance makes doing tasks feel easy...,General,NaN
2,What level of the hierarchy of needs is curren...,1.🧘 Physiological\n2.🔒 Safety \n3.💌 Social \n4...,General,NaN
3,"Aside from achieving peak performance, which o...",1.Reduced labor turnover and absenteeism.\n2.I...,Health Literacy,NaN
4,"I’m able to fulfill, meet, or exceed expectati...",1.Strongly Agree\n2.Agree\n3.Neutral\n4.Disagr...,Health Tracking,yes


### Clean question_catalogue

In [ ]:
#split answer column by \n
question_catalogue['answers_v1'] = question_catalogue['answers_v1'].apply(lambda x: x.split('\n'))

#remove blank and empty strings 
question_catalogue['answers_v1'] = question_catalogue['answers_v1'].apply(lambda x: [item.strip() for item in x if item.strip()])

# separate answers into different rows 
question_catalogue=question_catalogue.explode('answers_v1')

# separate number from text into different columns 
question_catalogue[['answer_number', 'answer_text']] = question_catalogue['answers_v1'].str.extract(r'(\d+)(.*)')

#remove trailing '.' from  answers 
question_catalogue['answer_text']=question_catalogue['answer_text'].str.strip('. !')

#remove trailing \r from questions 
question_catalogue['question_v1']=question_catalogue['question_v1'].str.rstrip('\r')

#change data type of the answer_number column 
question_catalogue['answer_number'] = question_catalogue['answer_number'].astype(int)

#change data type of the reverse_answer column 
question_catalogue.loc[:, 'reverse_answer? [yes/no]'] = question_catalogue['reverse_answer? [yes/no]'].astype(str)

#remove trailing spaces 

question_catalogue.loc[:, 'reverse_answer? [yes/no]'] = question_catalogue['reverse_answer? [yes/no]'].str.strip(' ')

In [ ]:
#check if there are null values, meaning the regex is wrong 
null_values = question_catalogue['answer_number'].isnull().sum() 
null_values

0

In [ ]:
# change name of question_v1 column
question_catalogue['question']=question_catalogue['question_v1']

# create another dataframe with only the columns we care about 
question_catalogue_clean=question_catalogue[['question',
                                            'answer_number',
                                            'answer_text',
                                            'type',
                                            'reverse_answer? [yes/no]']]
question_catalogue_clean.reset_index(inplace=True)

## Health tracking answer values 

In [ ]:
## SELECT HEALTH TRACKING AND BUILD THE LOGIC TO ASSING A VALUE DEPENDING ON THE ANSWER NUMBER,AND REVERSE SCALE AND THE SCALE OF 0 TO 10000

In [ ]:
filter=question_catalogue_clean['type']=='Health Tracking'

question_catalogue_clean[filter][['question','answer_number']].groupby('question').count()

## here we see that all health tracking questions have 5 answers 
## every answer goes from 1 to 5 
## if reverse=false 5=10000 and 1=0
## if reverse=true 1=10000 and 5=0

,answer_number
question,
How often are you able to guide your thoughts back to being present as soon as you get distracted?,5
Are you able to perform tasks free from distractions?,5
Do you feel that you are consuming foods with enough vitamins and minerals?,5
"How do you feel about turning down tasks that don’t offer anything towards your personal meaning and purpose, like turning down acquaintances who invite you to do things you don’t want to do, or removing activities that are distractions and don’t bring real value to your life from your schedule?",5
How often do you feel that the activities you carry out are aligned with your personal values?,5
How often do you set clear priorities in your life?,5
How often do you take breaks to take care of yourself?,5
I hold myself back during professional and personal experiences because I fear I might make situations worse.,5
"I know how to identify important activities that contribute to the meaning and purpose of my personal life, such as helping others when I can, donating objects or food to others in need, taking part in community work, and sharing information others may find useful for their own lives, among other things..\n",5


In [ ]:
#only health tracking has values in the reverse_answer column, there is no need to filter by type of question

for index, row in question_catalogue_clean.iterrows():

  reverse=question_catalogue_clean.loc[index,'reverse_answer? [yes/no]']

  if reverse=='no':
    question_catalogue_clean.loc[index,'answer_value']=(question_catalogue_clean.loc[index,'answer_number']-1)*2500
  if reverse=='yes':
    question_catalogue_clean.loc[index,'answer_value']=10000-(question_catalogue_clean.loc[index,'answer_number']-1)*2500


## if reverse=no 5=10000 and 1=0
## if reverse=yes 1=10000 and 5=0

if reverse is false that means that for the answer score, 5 = 10000; 4=7500; 3=5000; 2=2500 and 1 = 0. If the reverse is true the scale change direction, so 1=10000 and start decreasing in steps of 2500 until 5=0.


**If 1 is good > reverse = yes**




In [ ]:
# check answer values 

question_catalogue_clean[filter]

,index,question,answer_number,answer_text,type,reverse_answer? [yes/no],answer_value
18,4,"I’m able to fulfill, meet, or exceed expectati...",1,Strongly Agree,Health Tracking,yes,10000.0
19,4,"I’m able to fulfill, meet, or exceed expectati...",2,Agree,Health Tracking,yes,7500.0
20,4,"I’m able to fulfill, meet, or exceed expectati...",3,Neutral,Health Tracking,yes,5000.0
21,4,"I’m able to fulfill, meet, or exceed expectati...",4,Disagree,Health Tracking,yes,2500.0
22,4,"I’m able to fulfill, meet, or exceed expectati...",5,Strongly Disagree,Health Tracking,yes,0.0
...,...,...,...,...,...,...,...
203,43,Are you able to perform tasks free from distra...,1,Always,Health Tracking,yes,10000.0
204,43,Are you able to perform tasks free from distra...,2,Often,Health Tracking,yes,7500.0
205,43,Are you able to perform tasks free from distra...,3,Sometimes,Health Tracking,yes,5000.0
206,43,Are you able to perform tasks free from distra...,4,Seldom,Health Tracking,yes,2500.0


## Health literacy answer Values 

In [ ]:
filter=question_catalogue_clean['type']=='Health Literacy'

question_catalogue_clean[filter][['question','answer_number']].groupby('question').count()

## here we see that health literacy have a variable number of answer numbers 
## answer values can go from 0 to 1 . 1 being positive 
## there is no consistency in answer number and answer value. we will have to evaluate that case by case 

,answer_number
question,
"After discussing the first four simple techniques to achieve peak performance, which do you think will be most helpful for you now?",4
"Aside from achieving peak performance, which of the following do you think occurs in a workplace when employees are motivated?",4
Have you ever heard of the 4 pillars of peak performance?,2
Have you heard of SMART goals?,2
What do you think are the negative effects of participating in activities that contradict your values?,4
Which of the following vitamins and minerals are you interested in learning more about?,6
Why do you think it’s important for goals to be measurable?,3


In [ ]:
# manually add the answer_value for all health literacy questions:
filter=question_catalogue_clean['type']=='Health Literacy'

# do you want to reset the health_literacy values?
valid_inputs = ['Y', 'N','n','y']
user_input = input('do you want to reset all answer_values from health_literacy ?\n[Y/N]')

while user_input not in valid_inputs:
    print("Invalid input. Please enter Y or N.")
    user_input = input('do you want to reset all answer_values from health_literacy ?\n[Y/N]')

if user_input=='Y' or user_input=='y':
  question_catalogue_clean.loc[filter, 'answer_value'] = None

# Loop through each row in the DataFrame
print('input 0 or 1 for this questions-answers')
for index, row in question_catalogue_clean[filter].iterrows():
    #if there is already a value for answer, go next 
    if pd.isnull(question_catalogue_clean.loc[index, 'answer_value']):
      # Display the prompt with the value from the 'answer' column
      user_input = input(row['question']+'\n'+row['answer_text'] + ": ")
    
       # Update the 'value' column in the DataFrame
      question_catalogue_clean.loc[index, 'answer_value'] = user_input
    else:
      continue
print('All answers already have a value !')

do you want to reset all answer_values from health_literacy ?
[Y/N]Y
input 0 or 1 for this questions-answers
Aside from achieving peak performance, which of the following do you think occurs in a workplace when employees are motivated?
Reduced labor turnover and absenteeism: 1
Aside from achieving peak performance, which of the following do you think occurs in a workplace when employees are motivated?
Increased employee commitment: 1
Aside from achieving peak performance, which of the following do you think occurs in a workplace when employees are motivated?
Employees are open-minded about changes that happen within the organization: 1
Aside from achieving peak performance, which of the following do you think occurs in a workplace when employees are motivated?
I think all of the outcomes are possible: 1
Have you ever heard of the 4 pillars of peak performance?
👍: 1
Have you ever heard of the 4 pillars of peak performance?
👎: 0
What do you think are the negative effects of participating

In [ ]:
# check values 
question_catalogue_clean[filter].head()

,index,question,answer_number,answer_text,type,reverse_answer? [yes/no],answer_value
14,3,"Aside from achieving peak performance, which o...",1,Reduced labor turnover and absenteeism,Health Literacy,nan,1
15,3,"Aside from achieving peak performance, which o...",2,Increased employee commitment,Health Literacy,nan,1
16,3,"Aside from achieving peak performance, which o...",3,Employees are open-minded about changes that h...,Health Literacy,nan,1
17,3,"Aside from achieving peak performance, which o...",4,I think all of the outcomes are possible,Health Literacy,nan,1
59,13,Have you ever heard of the 4 pillars of peak p...,1,👍,Health Literacy,nan,1


## General Questions answer values 

In [ ]:
## all general questions should be 0 

In [ ]:
filter=question_catalogue_clean['type']=='General'

question_catalogue_clean.loc[filter,'answer_value']=0

## CHECKPOINT 

### Save cleaned question_catalogue

In [ ]:
#check entire dataframe of question_catalogue 
question_catalogue_clean.head()

,index,question,answer_number,answer_text,type,reverse_answer? [yes/no],answer_value
0,0,"First, I’m curious… Why did you decide to take this session?",1,I’m curious about the concept of peak performance,General,NaN,0.0
1,0,"First, I’m curious… Why did you decide to take this session?",2,I want to boost my knowledge on the topic,General,NaN,0.0
2,0,"First, I’m curious… Why did you decide to take this session?",3,I want to learn techniques to help me achieve peak performance,General,NaN,0.0
3,0,"First, I’m curious… Why did you decide to take this session?",4,I want to help other people become more aware of this concept,General,NaN,0.0
4,0,"First, I’m curious… Why did you decide to take this session?",5,All of these statements explain why I want to take this session,General,NaN,0.0


In [ ]:
# save dataframe (UNCOMENT THIS TO SAVE)
## question_catalogue_clean.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}question_catalogue_clean.csv')

### Read saved question_catalogue_clean

In [ ]:
# read saved question_catalogue_clean dataFrame 

# Specify the file path
file_path = (f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}question_catalogue_clean.csv')

# Read the CSV file into a DataFrame
question_catalogue_clean = pd.read_csv(file_path,index_col=0)

# Merge similar Questions using matching scores 


## questions from users 

#### Check similarity of questions inside logs




In [ ]:
#check that all questions from the all_data dataframe are unique using fuzzy matching

unique_questions=all_data[['question','answer']].groupby(['question']).count().sort_values('answer',ascending=False)
unique_questions.rename(columns={'answer': 'answer_num'}, inplace=True)
unique_questions.reset_index(inplace=True)

In [ ]:
unique_questions

,question,answer_num
0,"Okay, let’s jump right in",419
1,Is there any particular reason why you’re taking this session?,342
2,"<iframe width=""250"" height=""315"" src=""https://www.youtube.com/embed/Yi5yiMzEHRg?rel=0"" title=""YouTube video player"" frameborder=""0"" allow=""acceler...",250
3,Which of the following statements helps you best understand the definition of peak performance?,201
4,https://storage.googleapis.com/media.landbot.io/162855/channels/P2VYMOV0E7W3FUWAX3B95DRUNPFK39V1.png,182
...,...,...
100,Would you like to explore *Academly or take the WQ test?*,1
101,What would you like to do? 😃,1
102,https://wqtest.io/home Redirecting you in {timeout},1
103,Did you have any issues with the vocabulary?,1


In [ ]:
# Function to find similar string based on fuzzy matching and return similarity score
def find_similar_string(string, choices):
    choices = choices.drop(choices[choices == string].index)  # Exclude the string being evaluated
    best_match = process.extractOne(string, choices, scorer=fuzz.token_sort_ratio)
    if best_match is None or best_match[0] == string:
        return "", 0  # Return empty string and similarity score of 0
    else:
        return best_match[0], best_match[1]  # Return best match string and similarity score

# Apply fuzzy matching to create new columns for similar question, similarity score, and corresponding answer
result = unique_questions['question'].apply(lambda x: find_similar_string(x, unique_questions['question']))
unique_questions[['similar_question', 'similarity_score']] = pd.DataFrame(result.values.tolist(), index=result.index)

# Add corresponding answer column based on similar_question
unique_questions['similar_question_answer_num'] = unique_questions['similar_question'].map(unique_questions.set_index('question')['answer_num'])

In [ ]:
#add percentage of lost answers 
unique_questions['perc_dif_answers']=unique_questions['similar_question_answer_num']/unique_questions['answer_num']

#filter by similarity of the questions 
filter_similarity=unique_questions['similarity_score']>70

#filter by number of answers in the similar question
filter_lost_answers=unique_questions['similar_question_answer_num']>10

#filter by percentage of answers in the similar question
filter_lost_perc=unique_questions['perc_dif_answers']>0.1

#de-duplicate question - similar_question pairs 
filter_duplicate=unique_questions['answer_num']>unique_questions['similar_question_answer_num']


unique_questions[filter_similarity&filter_lost_answers&filter_lost_perc&filter_duplicate].sort_values('similar_question_answer_num',ascending=False)

,question,answer_num,similar_question,similarity_score,similar_question_answer_num,perc_dif_answers
19,"<iframe width=""560"" height=""315"" src=""https://www.youtube.com/embed/rtqS8IOELC8?rel=0"" title=""YouTube video player"" frameborder=""0"" allow=""acceler...",46,"<iframe width=""560"" height=""315"" src=""https://www.youtube.com/embed/rtqS8IOELC8"" title=""YouTube video player"" frameborder=""0"" allow=""accelerometer...",99,30,0.652174
11,Which of the following benefits would appeal to you the most?,65,Which of the following benefits of rest do you need most at the moment?,71,26,0.400000
4,https://storage.googleapis.com/media.landbot.io/162855/channels/P2VYMOV0E7W3FUWAX3B95DRUNPFK39V1.png,182,https://storage.googleapis.com/media.landbot.io/162855/channels/O763ZAOMFAEY6J6SWXQ0DZWIEA4OCBRM.png,76,24,0.131868


##### make dict with replacements
If no replacements are needed leave the dictionary empty 


In [ ]:
# Format:
"""
replacements:{'similar_question':'question',
              'similar_question':'question'}
"""
replacements={''}

In [ ]:
## create a new dataframe for the cleaned data 
all_data_cleaned=all_data

## replace the questions using the dict :
all_data_cleaned['question']=all_data['question'].replace(replacements)


#### check similarity with questions in the catalogue 

In [ ]:
## make dataframe 
# Get unique questions from all_data_cleaned
unique_questions_all_data = pd.DataFrame({'question': all_data_cleaned['question'].unique()})

# Get unique questions from question_catalogue_clean
unique_questions_question_catalogue = pd.DataFrame({'question': question_catalogue_clean['question'].unique()})

In [ ]:
# Function to find the most approximate match from df2 for a given question
def find_approximate_match(question, df2_questions):
    exact_matches = [q for q in df2_questions if q != question]  # Exclude exact matches
    best_match = process.extractOne(question, exact_matches, scorer=fuzz.token_sort_ratio)
    return best_match[0], best_match[1]

# Load the two DataFrames
df1 = unique_questions_all_data  # DataFrame 1 with questions
df2 = unique_questions_question_catalogue # DataFrame 2 with questions

# Get a list of questions and matching scores from df2
df2_questions = df2['question'].tolist()

# Apply fuzzy matching to find the most approximate match from df2 for each question in df1
df1[['approximate_match', 'similarity_score']] = df1['question'].apply(lambda x: pd.Series(find_approximate_match(x, df2_questions)))

# Create a new DataFrame with questions from df1, their most approximate matches from df2, and matching scores
result_df = pd.DataFrame({'question_df1': df1['question'], 'approximate_match_df2': df1['approximate_match'], 'similarity_score': df1['similarity_score']})


SyntaxError: ignored

In [ ]:
result_df

,question_df1,approximate_match_df2,similarity_score
0,"Okay, let’s jump right in",I know my purpose.,39
1,Is there any particular reason why you’re taking this session?,"First, I’m curious… Why did you decide to take this session?",58
2,"<iframe width=""250"" height=""315"" src=""https://www.youtube.com/embed/Yi5yiMzEHRg?rel=0"" title=""YouTube video player"" frameborder=""0"" allow=""acceler...",Have you already had the opportunity to picture some of the techniques you learned in the last module and think about how they could be integrated...,41
3,Which of the following statements helps you best understand the definition of peak performance?,Which of the following statements from the video did you find most helpful for you to understand peak performance?,78
4,https://storage.googleapis.com/media.landbot.io/162855/channels/P2VYMOV0E7W3FUWAX3B95DRUNPFK39V1.png,I make goals that are important to me and that are connected to each other.,36
...,...,...,...
100,I found all of these statements helpful,"From what we’ve discussed so far, which of the following statements was most helpful to you?",54
101,Wow! Look how far we have come and how much we have explored together! 🚀💪,Approximately how much water do you consume per day?,47
102,What would you like to do? 😃,How often do you make time to exercise?,51
103,Would you like to explore *Academly or take the WQ test?*,Would you like to look at any of the other techniques to improve your sleep?,59


In [ ]:
#filter by similarity of the questions 
filter_similarity=(result_df['similarity_score']>82)


pd.set_option('display.max_colwidth', 150)

result_df[filter_similarity].sort_values('similarity_score',ascending=False)


,question_df1,approximate_match_df2,similarity_score
25,Which of the following techniques are you most interested in at the moment?,Which of the following techniques we have discussed in this module are you most interested in?,83
64,Which of the following techniques we have discussed in this module are you most interested in?,Which of the following techniques are you most interested in at the moment?,83


##### make the replacements
The questions in the dataframe above are the ones that will be replaced.

**Change filter_similarity until the dataframe only has the questions we want replaced**


In [ ]:
# select replacements :
replacements_df=result_df[filter_similarity].sort_values('similarity_score',ascending=False)

# Convert DataFrame to dictionary
replacements = dict(zip(replacements_df['question_df1'], replacements_df['approximate_match_df2']))

## replace the questions using the dict :
all_data_cleaned['question']=all_data['question'].replace(replacements)

In [ ]:
#@title
#check dataframe 
all_data_cleaned.head()

,created_at,question,user_number,answer,user_mail
0,2022/09/05 10:25:47,"Okay, let’s jump right in",203343214,Sounds good,jsomlock17@gmail.com
1,2022/09/05 10:25:56,Is there any particular reason why you’re taking this session?,203343214,All of these statements explain why I want to take this session,jsomlock17@gmail.com
2,2022/09/05 10:26:03,"<iframe width=""250"" height=""315"" src=""https://www.youtube.com/embed/Yi5yiMzEHRg?rel=0"" title=""YouTube video player"" frameborder=""0"" allow=""acceler...",203343214,Continue,jsomlock17@gmail.com
3,2022/09/05 10:26:28,Which of the following statements helps you best understand the definition of peak performance?,203343214,"Peak performance means being confident, creative, focused, and recovered from daily functions",jsomlock17@gmail.com
4,2022/09/05 10:26:40,https://storage.googleapis.com/media.landbot.io/162855/channels/P2VYMOV0E7W3FUWAX3B95DRUNPFK39V1.png,203343214,Continue,jsomlock17@gmail.com


cleaned dataframe for all_data 

In [ ]:
#check the dataframe 
all_data_cleaned

,created_at,question,user_number,answer,user_mail
0,2022/09/05 10:25:47,"Okay, let’s jump right in",203343214,Sounds good,jsomlock17@gmail.com
1,2022/09/05 10:25:56,Is there any particular reason why you’re taking this session?,203343214,All of these statements explain why I want to take this session,jsomlock17@gmail.com
2,2022/09/05 10:26:03,"<iframe width=""250"" height=""315"" src=""https://www.youtube.com/embed/Yi5yiMzEHRg?rel=0"" title=""YouTube video player"" frameborder=""0"" allow=""acceler...",203343214,Continue,jsomlock17@gmail.com
3,2022/09/05 10:26:28,Which of the following statements helps you best understand the definition of peak performance?,203343214,"Peak performance means being confident, creative, focused, and recovered from daily functions",jsomlock17@gmail.com
4,2022/09/05 10:26:40,https://storage.googleapis.com/media.landbot.io/162855/channels/P2VYMOV0E7W3FUWAX3B95DRUNPFK39V1.png,203343214,Continue,jsomlock17@gmail.com
...,...,...,...,...,...
3394,2021/07/28 23:21:57,"🦺 My workplace provides adequate infrastructure, like ergonomic chairs, desks, and safety equipment that help me keep motivated at work.",117638351,👍,je.suis.paheli03@gmail.com
3395,2021/07/28 23:22:07,"📚 My workplace provides resources that engage my mind, that help me improve my abilities and strengthen my skills so that I can achieve peak perfo...",117638351,Sometimes,je.suis.paheli03@gmail.com
3396,2021/07/28 23:22:21,Would you be interested in taking a look at the session on _Obtaining Work-Life Balance_ after we finish with this one?,117638351,👍,je.suis.paheli03@gmail.com
3397,2021/07/28 23:22:56,"From what we’ve discussed so far, which of the following statements was most helpful to you?",117638351,🏃‍♀️ Being motivated is a vital factor for achieving peak performance,je.suis.paheli03@gmail.com


## Answers from users 

#### check similarity of answers inside logs 

In [ ]:
#check that all answers from the all_data_cleaned dataframe are unique using fuzzy matching

unique_answers=all_data_cleaned[['question','answer']].groupby(['answer']).count().sort_values('question',ascending=False)
unique_answers.rename(columns={'question': 'question_num'}, inplace=True)
unique_answers.reset_index(inplace=True)

In [ ]:
unique_answers

,answer,question_num
0,Continue,842
1,Sounds good,428
2,👍,253
3,I want to learn techniques to help me achieve peak performance,198
4,I found all of these statements helpful,130
...,...,...
132,The opportunity for spiritual development,1
133,The recommendations were good for me,1
134,The recommendations weren’t the problem,1
135,The vocabulary wasn’t the problem,1


In [ ]:
# Function to find similar string based on fuzzy matching and return similarity score
def find_similar_string(string, choices):
    choices = choices.drop(choices[choices == string].index)  # Exclude the string being evaluated
    best_match = process.extractOne(string, choices, scorer=fuzz.token_sort_ratio)
    if best_match is None or best_match[0] == string:
        return "", 0  # Return empty string and similarity score of 0
    else:
        return best_match[0], best_match[1]  # Return best match string and similarity score

# Apply fuzzy matching to create new columns for similar question, similarity score, and corresponding answer
result = unique_answers['answer'].apply(lambda x: find_similar_string(x, unique_answers['answer']))
unique_answers[['similar_answer', 'similarity_score']] = pd.DataFrame(result.values.tolist(), index=result.index)

# Add corresponding answer column based on similar_question
unique_answers['similar_answer_question_num'] = unique_answers['similar_answer'].map(unique_answers.set_index('answer')['question_num'])

In [ ]:

#add percentage of lost questions
unique_answers['perc_dif_questions']=unique_answers['similar_answer_question_num']/unique_answers['question_num']

#filter by similarity of the questions 
filter_similarity=unique_answers['similarity_score']>80

#filter by number of answers in the similar question
filter_lost_answers=unique_answers['similar_answer_question_num']>10

#filter by percentage of answers in the similar question
filter_lost_perc=unique_answers['perc_dif_questions']>0.1

#de-duplicate question - similar_question pairs 
filter_duplicate=unique_answers['question_num']>unique_answers['similar_answer_question_num']


unique_answers[filter_similarity&filter_lost_answers&filter_lost_perc&filter_duplicate].sort_values('similar_answer_question_num',ascending=False)

,answer,question_num,similar_answer,similarity_score,similar_answer_question_num,perc_dif_questions
2,👍,253,👎,100,104,0.411067
12,Strongly agree,49,Strongly disagree,90,28,0.571429


##### **make a dict with the replacements**
If no replacements are needed leave the dictionary empty 


In [ ]:
# Format:
"""
replacements:{'similar_answer':'answer',
              'similar_answer':'answer'}
"""
replacements={'':''}

In [ ]:
## replace the questions using the dict :
all_data_cleaned['question']=all_data['question'].replace(replacements)

In [ ]:
#check dataframe 
all_data_cleaned.head()

,created_at,question,user_number,answer,user_mail
0,2022/09/05 10:25:47,"Okay, let’s jump right in",203343214,Sounds good,jsomlock17@gmail.com
1,2022/09/05 10:25:56,Is there any particular reason why you’re taking this session?,203343214,All of these statements explain why I want to take this session,jsomlock17@gmail.com
2,2022/09/05 10:26:03,"<iframe width=""250"" height=""315"" src=""https://www.youtube.com/embed/Yi5yiMzEHRg?rel=0"" title=""YouTube video player"" frameborder=""0"" allow=""acceler...",203343214,Continue,jsomlock17@gmail.com
3,2022/09/05 10:26:28,Which of the following statements helps you best understand the definition of peak performance?,203343214,"Peak performance means being confident, creative, focused, and recovered from daily functions",jsomlock17@gmail.com
4,2022/09/05 10:26:40,https://storage.googleapis.com/media.landbot.io/162855/channels/P2VYMOV0E7W3FUWAX3B95DRUNPFK39V1.png,203343214,Continue,jsomlock17@gmail.com


#Join dataframes: all_data to question_catalogue 

## Filter all_data 

In [ ]:
#only keep questions from the log that are in the question catalogue 
filter=all_data_cleaned['question'].isin(question_catalogue_clean['question'].tolist())
all_data_cleaned=all_data_cleaned[filter]

NameError: ignored

## Finding best match for answers 



In [ ]:
#This block of code works like this :

# For each row in the all_data_cleaned dataframe : ( wich is where we have the question logs and users data )
# Select the question and the answer given by the user
# Filters the question_catalogue to only show the answers for that question
# Search for an exact match between the answer given by the user and the posible answers for that question in the question_catalogue
# If a match is found : it writes it to the 'best_match' column 
# if a match is not found : we search for the best possible match in that subset  of posible answers to the question
# then  write the best match, together with its matching score  to 'best_match' and 'match_score'column

# at the end we need to filter the dataframe to save only high matching scores and exact matches.

# Create columns 
all_data_cleaned['best_match'] = None
all_data_cleaned['match_score'] = None

# Loop through each row in the DataFrame
for index, row in all_data_cleaned.iterrows():

    # Function to find the closest match
    def find_closest_match(query, choices):
        best_match = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)
        return best_match

    # Select the question value
    question = all_data_cleaned.loc[index, 'question']

    # Select the answer value
    answer = all_data_cleaned.loc[index, 'answer']

    # Create filter for the question catalogue
    filter = question_catalogue_clean['question'] == question

    # Filter the question catalogue
    question_catalogue_filtered = question_catalogue_clean[filter]

    # Perform the exact match search
    for index_q, row_q in question_catalogue_filtered.iterrows():
      if answer==question_catalogue_filtered.loc[index_q,'answer_text']:
        all_data_cleaned.loc[index, 'best_match'] = question_catalogue_filtered.loc[index_q, 'answer_text']
        break  # Exit the loop if a match is found

    # Check if any exact matches were found
    if all_data_cleaned.loc[index, 'best_match']==None:

      # Select the best match
      best_match = find_closest_match(answer, question_catalogue_filtered['answer_text'].tolist())

      # Write best match and matching score to DataFrame
      all_data_cleaned.loc[index, 'best_match'] = best_match[0]
      all_data_cleaned.loc[index, 'match_score'] = best_match[1]

all_data_cleaned

<ipython-input-140-3a1cf9ce5193>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_cleaned['best_match'] = None
<ipython-input-140-3a1cf9ce5193>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_cleaned['match_score'] = None


,created_at,question,user_number,answer,user_mail,best_match,match_score
3,2022/09/05 10:26:28,Which of the following statements helps you best understand the definition of peak performance?,203343214,"Peak performance means being confident, creative, focused, and recovered from daily functions",jsomlock17@gmail.com,"Peak performance means being confident, creative, focused, and recovered from daily functions",None
12,2022/08/29 17:55:27,Which of the following statements helps you best understand the definition of peak performance?,201669059,"Peak performance means being confident, creative, focused, and recovered from daily functions",pranitaaa11@gmail.com,"Peak performance means being confident, creative, focused, and recovered from daily functions",None
32,2022/08/15 04:36:07,Which of the following benefits of rest do you need most at the moment?,198402979,Refueled energy,oliwiacgbartosiak@gmail.com,Refueled energy,None
33,2022/08/15 04:36:18,How often do you take breaks to take care of yourself?,198402979,Sometimes,oliwiacgbartosiak@gmail.com,Sometimes,None
36,2022/08/15 04:37:11,Which of the following self-care practices do you carry out the most?,198402979,Moisturizing skin,oliwiacgbartosiak@gmail.com,Going for a walk. 🚶,42
...,...,...,...,...,...,...,...
3389,2021/07/28 23:18:40,I hold myself back during professional and personal experiences because I fear I might make situations worse.,117638351,Strongly agree,je.suis.paheli03@gmail.com,Strongly Agree,100
3390,2021/07/28 23:19:05,Are there things you’d like to change about your work habits?,117638351,"📅 Yes, I’d like to be more punctual/have better organizational skills",je.suis.paheli03@gmail.com,"📅 Yes, I’d like to be more punctual/have better organizational skills",None
3394,2021/07/28 23:21:57,"🦺 My workplace provides adequate infrastructure, like ergonomic chairs, desks, and safety equipment that help me keep motivated at work.",117638351,👍,je.suis.paheli03@gmail.com,Strongly Agree,0
3395,2021/07/28 23:22:07,"📚 My workplace provides resources that engage my mind, that help me improve my abilities and strengthen my skills so that I can achieve peak perfo...",117638351,Sometimes,je.suis.paheli03@gmail.com,Sometimes,None


###check that the matches were done correctly 

In [ ]:
df=all_data_cleaned[['question','answer','best_match','match_score']]
unique_df = df.drop_duplicates(subset=['question', 'answer', 'best_match','match_score']).sort_values('match_score')
unique_df.head(20)

,question,answer,best_match,match_score
1254,Do you feel that you are consuming foods with enough vitamins and minerals?,👍,Strongly Agree,0
76,"🦺 My workplace provides adequate infrastructure, like ergonomic chairs, desks, and safety equipment that help me keep motivated at work.",👍,Strongly Agree,0
154,"🦺 My workplace provides adequate infrastructure, like ergonomic chairs, desks, and safety equipment that help me keep motivated at work.",👎,Strongly Agree,0
354,Do you feel that you are consuming foods with enough vitamins and minerals?,👎,Strongly Agree,0
2737,"Aside from achieving peak performance, which of the following do you think occurs in a workplace when employees are motivated?",START AGAIN,Reduced labor turnover and absenteeism,29
626,Which of the following benefits of rest do you need most at the moment?,Confidence when making decisions,Restored body tissues,34
1458,Which of the following benefits of rest do you need most at the moment?,The opportunity for spiritual development,Stimulated creativity,39
2726,Which of the following benefits of rest do you need most at the moment?,Increased self-assurance,Restored body tissues,40
1112,What level of the hierarchy of needs is currently motivating you the most?,START AGAIN,🔝 Self-actualization,41
1890,Which of the following benefits of rest do you need most at the moment?,Reduced instances of stress,Restored body tissues,42


## Setting Matching scores threshold 

**Change filter_match_score until only the answers we want replaced are left in the filtered dataframe**

In [ ]:
## match score filter 
filter_match_score=all_data_cleaned['match_score']<=65 ##CHANGE THIS FILTER AND CHECK THE RESULTING DATAFRAME 


#check the dataframe 
all_data_cleaned[~filter_match_score]


### Apply filter to dataframe 

In [ ]:
#filter dataframe 
all_data_cleaned=all_data_cleaned[~filter_match_score].copy()

## join dataframes 

In [ ]:
## join dataframes

# Join the DataFrames using two columns as the join keys
merged_df = pd.merge(all_data_cleaned,
                     question_catalogue_clean, 
                     left_on=['question','best_match'],
                     right_on=['question','answer_text'],
                     how='inner')

In [ ]:
final_dataframe=merged_df[['created_at',
                          'question',
                          'user_number',
                          'answer',
                          'user_mail',
                          'answer_value']]

final_dataframe

,created_at,question,user_number,answer,user_mail,answer_value
0,2022/08/15 04:37:18,Now which of the following would you be interested in trying next?,198402979,Spending quality time with loved ones,oliwiacgbartosiak@gmail.com,0.0
1,2022/01/15 16:42:06,Now which of the following would you be interested in trying next?,151399570,Spending quality time with loved ones,happypooja65@gmail.com,0.0
2,2022/08/15 04:38:50,Which of the following ways of getting better sleep would you want to try?,198402979,Creating a bed routine,oliwiacgbartosiak@gmail.com,0.0
3,2022/10/28 15:14:53,Which of the following ways of getting better sleep would you want to try?,216763477,Creating a bed routine,hemavg2123@gmail.com,0.0
4,2022/10/28 15:15:40,Which of the following ways of getting better sleep would you want to try?,216763477,Creating a bed routine,hemavg2123@gmail.com,0.0
...,...,...,...,...,...,...
239,2021/10/30 09:11:15,I know my purpose.,136715510,Strongly disagree,nicegirlyas.yb@gmail.com,0.0
240,2022/01/15 16:41:50,Which of the following self-care practices do you carry out the most?,151399570,Watching a movie,happypooja65@gmail.com,0.0
241,2021/11/17 23:34:30,Which of the following self-care practices do you carry out the most?,140053703,Watching a movie,jakehandling@gmail.com,0.0
242,2021/11/09 03:06:33,Which of the following self-care practices do you carry out the most?,138444351,Watching a movie,aislingrodgers74@gmail.com,0.0


#Save final dataframe 

#### overwrite
* Its the first time runing the script
* We want to make the dataframe again
when the data here is of all the log files (we process the entire folder) and 

In [ ]:
""" DELETE THIS ROW IF YOU WANT TO OVERRWRITE 

final_dataframe.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv')

""" ## DELETE THIS ROW IF YOU WANT TO OVERRWRITE 

" DELETE THIS ROW IF YOU WANT TO OVERRWRITE \n\n\nfinal_dataframe.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv')\n\n"

#### Append 
* should be used when we want to add data partially ( for example we already have a data frame made with this script  and we get new data from landbot)


In [ ]:
""" ## DELETE THIS ROW IF YOU WANT TO APPEND


# Check if the CSV file exists
if os.path.isfile(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv'):
    # Read the existing CSV file into a dataframe
    existing_dataframe = pd.read_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv')
else:
    # Print an error message and exit the script
    print(f"File not found , is this the first time running this script for this folder ?")
    exit()

# Append the final_dataframe to the existing dataframe
appended_dataframe = existing_dataframe.append(final_dataframe)

# Write the appended dataframe to the CSV file
appended_dataframe.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv')

""" ## DELETE THIS ROW IF YOU WANT TO APPEND

' ## DELETE THIS ROW IF YOU WANT TO APPEND\n\n\n# Check if the CSV file exists\nif os.path.isfile(f\'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv\'):\n    # Read the existing CSV file into a dataframe\n    existing_dataframe = pd.read_csv(f\'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv\')\nelse:\n    # Print an error message and exit the script\n    print(f"File not found , is this the first time running this script for this folder ?")\n    exit()\n\n# Append the final_dataframe to the existing dataframe\nappended_dataframe = existing_dataframe.append(final_dataframe)\n\n# Write the appended dataframe to the CSV file\nappended_dataframe.to_csv(f\'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv\')\n\n'